# 音檔靜音切割方法

In [4]:
from pydub.silence import split_on_silence
from pydub import AudioSegment
import os
import librosa
def sound_split(filepath):
    sound = AudioSegment.from_wav(filepath)
    dBFS = sound.dBFS # 指定最大聲音
    print(dBFS)
    chunks = split_on_silence(sound, 
        min_silence_len = 1000,
        silence_thresh = dBFS-16)
    return chunks



In [5]:
# 單檔整理
path = r"F:\python_coding\sound\wavfile\ofAngryCatSound.wav"
path_to = r"F:\python_coding\sound\wavfile"
data = sound_split( path )
if  not os.path.exists(path_to): # 判斷目錄是否存在，不存在就建立檔案
    os.mkdir(path_to)
for j in range(len(data)):
    data[j].export( f"{path_to}/{j}.wav", format="wav")

-18.886473157508124


KeyboardInterrupt: 

In [96]:
# 多檔整理
path = "F:/python_coding/sound/wavfile/need_follow"
path_to = r"F:/python_coding/sound/wavfile/cut_need_follow"
filename= os.listdir(path)

if  not os.path.exists(path_to): # 判斷目錄是否存在，不存在就建立檔案
    os.mkdir(path_to)

for i in range(len(filename)):
    data = sound_split( path+"/"+filename[i])
    for j in range(len(data)):
        data[j].export( f"{path_to}/{i}_{j}.wav", format="wav")

-30.986164044413698
-31.19524369292424
-35.100549030730114
-29.57642079195113
-37.619453295979724
-36.64805793441736
-36.225988362398375
-21.1121058514302
-40.53798436918547


In [9]:
path = "F:/python_coding/sound/wavfile/estrus"
filename= os.listdir(path)
size = os.path.getsize( path+"/"+filename[0])
print(f"file:{filename[0]} size: {size} and sizetype: {type(size)}")

file:Cat in Heat (Estrus).wav size: 27840590 and sizetype: <class 'int'>


# 音檔固定長度切割

In [9]:
import os
import wave
import numpy as np
import pylab as plt
import librosa
from pydub import AudioSegment
from ffmpy import FFmpeg


def create_data(path_from,path_to,item_len = 2000):
    if path_from.endswith('.wav'):# 輸入的資料來源如果是.wav
        files = [path_from]
    else: # 輸入的資料來源如果是資料夾
        files = os.listdir(path_from)
        files = [f"{path_from}/{ f}" for f in files if f.endswith('.wav')]
    print(files[0])
    if  not os.path.exists(path_to): # 判斷目錄是否存在，不存在就建立檔案
        os.mkdir(path_to)
    
    for file in files:
        file_name = "".join((file.split('/').pop()).split("_")[:3])
        print(file_name)
        file_name = file_name.split('.')[0]
        audio = AudioSegment.from_file(file)
        audio_len = len(audio)

        steps = audio_len // item_len+1
        for step in range(steps):
            item_audio = audio[step * item_len:( (step + 1) * item_len if (step + 1) * item_len < audio_len else audio_len )]
            save_audio = item_audio + AudioSegment.silent(item_len - len(item_audio))
            new_path = f"{path_to}/{file_name}_{step}.wav"
            print( new_path)
            save_audio.export( new_path)
    
    

In [10]:
path_from = "F:/python_coding/sound/call_attention-20200616T083858Z-001/purring"
path_to = "F:/python_coding/sound/call_attention-20200616T083858Z-001/cut_purring"
create_data(path_from , path_to)

F:/python_coding/sound/call_attention-20200616T083858Z-001/purring/Cat Purring Sound Effect.wav
Cat Purring Sound Effect.wav
F:/python_coding/sound/call_attention-20200616T083858Z-001/cut_purring/Cat Purring Sound Effect_0.wav
F:/python_coding/sound/call_attention-20200616T083858Z-001/cut_purring/Cat Purring Sound Effect_1.wav
F:/python_coding/sound/call_attention-20200616T083858Z-001/cut_purring/Cat Purring Sound Effect_2.wav
F:/python_coding/sound/call_attention-20200616T083858Z-001/cut_purring/Cat Purring Sound Effect_3.wav
F:/python_coding/sound/call_attention-20200616T083858Z-001/cut_purring/Cat Purring Sound Effect_4.wav
F:/python_coding/sound/call_attention-20200616T083858Z-001/cut_purring/Cat Purring Sound Effect_5.wav
F:/python_coding/sound/call_attention-20200616T083858Z-001/cut_purring/Cat Purring Sound Effect_6.wav
F:/python_coding/sound/call_attention-20200616T083858Z-001/cut_purring/Cat Purring Sound Effect_7.wav
F:/python_coding/sound/call_attention-20200616T083858Z-001/

# 其他

In [11]:
import os
import wave
import numpy as np
import pylab as plt
import librosa
from pydub import AudioSegment

path = r"F:/python_coding/sound/wavfile/all_cut_sound"
files = os.listdir(path)
files = [path + "/" + f for f in files if f.endswith('.wav')]
CutTime = 1 #单位长度4s
def create_data(path="F:/catsound/excmple", extension = "wav"):
    item_len = 2000
    
    for file in [x for x in os.listdir(path) if extension in x]:
        real_path = path+"/" + file
        audio = AudioSegment.from_file(real_path)
        audio_len = len(audio)

        steps = audio_len // item_len
        for step in range(max(1, steps)):
            item_audio = audio[step * item_len: (step + 1) * item_len]
            save_audio = item_audio + AudioSegment.silent(item_len - len(item_audio))
            
#             save_audio.export(path+"/data/" + str(step) + file, format=extension)
            save_audio.export("F:/python_coding/sound/wavfile/all_cut_sound_1s/" + str(step) + file, format=extension)
            
   

create_data(path)

In [85]:
from pydub import AudioSegment
import os
import numpy as np
import scipy.io.wavfile as wav
import random
import librosa
from sklearn.model_selection import train_test_split
import gc
SAMPLING_RATE = 44100
MFCC_NUM = 20
MFCC_MAX_LEN = 2000
def wav2mfcc(wave, max_len=2000):
#     mfcc = librosa.feature.mfcc(wave, sr=16000)
    mfcc = librosa.feature.mfcc(wave, n_mfcc=MFCC_NUM, sr=SAMPLING_RATE)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

def create_test_data(path="F:/catsound/excmple", extension = "wav"):
#     wave, sr = librosa.load(path, mono=True, sr=None)
    mfcc_list = []
    audio, sr = librosa.load(path, mono=True, sr=None)
# Get number of samples for 2 seconds; replace 2 by any number
    buffer = 2 * sr
    samples_total = len(audio)
    samples_wrote = 0
    counter = 1
    while samples_wrote < samples_total:
        #check if the buffer is not exceeding total samples 
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote
        block = audio[samples_wrote : (samples_wrote + buffer)]
        mfcc = wav2mfcc(block,MFCC_MAX_LEN)
        mfcc_list.append(mfcc)
        counter += 1
        samples_wrote += buffer
    mfcc_list = np.array(mfcc_list)
    mfcc_list = mfcc_list.reshape(mfcc_list.shape[0], MFCC_NUM, MFCC_MAX_LEN, 1)
    return mfcc_list
        
# test_one_list = create_test_data("F:/python_coding/sound/wavfile/cut_angry/cat_angry (876).wav")
test_one_list = create_test_data("F:/python_coding/sound/来听猫叔打呼噜吧, 非常的治愈哦 标清.wav")
# for i in test_one_list:
#     ipd.Audio(i)

In [82]:
test_one_list.shape

(2, 20, 2000, 1)

In [84]:
from keras.models import load_model

# 載入模型
model = load_model('F:/python_coding/sound/model.h5')

In [ ]:
from keras.utils import to_categorical,plot_model
plot_model(model, to_file='F:/Mypython/sound/model.png',show_shapes = True)

In [86]:
Y_testnas = model.predict(test_one_list)
around1 = np.around(Y_testnas)
around1

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 0.]], dtype=float32)